In [1]:
import music21
from music21 import stream, midi, scale, converter
import numpy as np
import subprocess 

In [4]:
class S1987:
    def __init__(self, lilypondPath, yourfilepath):
        self.lilypondPath = lilypondPath
        self.yourfilepath = yourfilepath
    def __str__(self):
        return """Music Variation Maker V.2: Work on progress (0 ^ 0)
        This package is building for piano user only.
        Function in this package
        1. ???
        2. ???
        3. ???
        
        """
    def note_converter(self, midi_note):
        """
        This function convert single midi note value to readable musical note.
        Important parameter:
        midi_note: midi note value;
                   60 ----> C4
        """
        if midi_note == '<REST>':
            return '<REST>'
        else:
            if 0 <= midi_note <= 127:
                pitches = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

                octave = (midi_note // 12) - 1
                pitch_index = midi_note % 12

                pitch = pitches[pitch_index]
                return f"{pitch}{octave}"
            else:
                return "Invalid MIDI note value. Must be between 0 and 127."
    def le(self, t, xyz, *args, **kwargs):
        """
        ********************************************************************************
        "I didn't have any knowledge about Chaotic System" you say,
        Nah Just use this one and change to any chaotic system you want later.
        Chaotic Trajectory parameter:
        d: 10 
        r: 28 
        b: 8/3 
        ********************************************************************************
        !!! This function relate with rk4 !!!
        """
        d, r, b = kwargs['d'], kwargs['r'], kwargs['b']
        x = xyz[0]
        y = xyz[1]
        z = xyz[2]
        xdot = d*(y - x)
        ydot = x*(r - z) - y
        zdot = x*y - b*z
        return np.array([xdot, ydot, zdot])
    def rk4(self, time=None, ini=None, f=le, h=0.01, *args, **kwargs):
        """
        ********************************************************************************
        This function can approximate solution of Chaotic system, but this version
        isn't support import new chaotic solution from outsource.
        Important parameter:
        time: The list of time you want from start to finish;
              [0, 40] ---> (start, end)
        ini: The seed for trajectory;
             np.array([1,1,1]) ----> [x, y, z]
        h:   step size;
             0.01, 0.001, 0.0001
        ********************************************************************************
        !!! This function relate with music_variation_generator !!!
        """
        t = np.arange(time[0], time[1]+h, h)
        row = len(t)
        try:
            var = len(ini)
        except TypeError:
            var = 1
        x = np.zeros((row, var))
        x[0, :] = ini
        for j in range(0, len(t)-1):
            k1 = f(self, t[j], x[j], *args, **kwargs)
            k2 = f(self, t[j] + 0.5*h, x[j] +0.5*h*k1, *args, **kwargs)
            k3 = f(self, t[j] + 0.5*h, x[j] + 0.5*h*k2, *args, **kwargs)
            k4 = f(self, t[j] + h, x[j] + h*k3, *args, **kwargs)
            x[j+1, :] = x[j] + (h/6)*(k1 + 2*k2 + 2*k3 + k4)
        return t, x
    def note_detector(self, Room_num_list1, Room_num_list2, melody_list, bass_list, nv_melody_list, nv_bass_list):
        melody_report_stored_list = []
        note_index1 = 0
        for i in range(len(Room_num_list1)):
            for j in range(Room_num_list1[i]):
                if melody_list[note_index1] != nv_melody_list[note_index1]:
                    melody_report_stored_list.append([i+1, j+1, melody_list[note_index1], nv_melody_list[note_index1]])
                note_index1 += 1
        
        bass_report_stored_list = []
        note_index2 = 0
        for i in range(len(Room_num_list2)):
            for j in range(Room_num_list2[i]):
                if bass_list[note_index2] != nv_bass_list[note_index2]:
                    bass_report_stored_list.append([i+1, j+1, bass_list[note_index2], nv_bass_list[note_index2]])
                note_index2 += 1
        
        return melody_report_stored_list, bass_report_stored_list
    def clean_report_list(self, report_list):
        for row in report_list:
            for j in range(2, len(row)):
                if row[j] == '<REST>':
                    row[j] = 'REST'    
                elif isinstance(row[j], str) and '#' in row[j]:
                    row[j] = row[j].replace('#', r'♯')
                elif isinstance(row[j], list):
                    for k in range(len(row[j])):
                        if '#' in row[j][k]:
                            row[j][k] = row[j][k].replace('#', r'♯')
        return report_list
    def report_maker(self, report_melody, report_bass, report_name, cleaner=clean_report_list):
        report_list1 = report_melody
        report_list2 = report_bass
        report_list1 = cleaner(self, report_list=report_list1)
        report_list2 = cleaner(self, report_list=report_list2)
        
        latex_file = report_name + '.tex'

        with open(latex_file, 'w', encoding='utf-8') as f:
            f.write(r'\documentclass{article}' + '\n')
            f.write(r'\usepackage{longtable}' + '\n')
            f.write(r'\usepackage[a4paper,margin=1in]{geometry}' + '\n')
            f.write(r'\usepackage{fontspec}' + '\n')
            f.write(r'\setmainfont{Arial Unicode MS}' + '\n')
            
            f.write(r'\title{' + report_name + '}' + '\n')
            
            f.write(r'\begin{document}' + '\n')
            
            f.write(r'\maketitle' + '\n')
            f.write(r'\section*{About Report}' + '\n')
            f.write(r'This PDF is designed to show the position of the original notes and the notes that have been changed in the MIDI file. \\' + '\n')
            f.write(r'Warning: Because of a sharp symbol error in Python, we decided to use ♯ for the sharp musical symbol. Which mean you will need Arial Unicode MS font for compile this PDF' + '\n')

            f.write(r'\section*{Melody Part Result}' + '\n')
            f.write(r'\begin{longtable}{|c|c|c|c|}' + '\n')
            f.write(r'\hline' + '\n')
            f.write(r'Bar & Note Position & Original Note & New Variation Note \\ \hline' + '\n')
            f.write(r'\endfirsthead' + '\n')
            f.write(r'\multicolumn{4}{c}{{\tablename\ \thetable{} Melody Part Result}} \\' + '\n')
            f.write(r'\hline' + '\n')
            f.write(r'Bar & Note Position & Original Note & New Variation Note \\ \hline' + '\n')
            f.write(r'\hline' + '\n')
            f.write(r'\endhead' + '\n')
            f.write(r'\hline' + '\n')
            f.write(r'\multicolumn{4}{r}{next page}' + '\n')
            f.write(r'\endfoot' + '\n')
            f.write(r'\hline' + '\n')
            f.write(r'\endlastfoot' + '\n')
            
            room_check1 = 1
            room_repeat_count1 = 0
            for row in report_list1:
                room = row[0]
                no_repeat_room = " "
                location = row[1]
                original_note = row[2]
                new_variation_note = row[3]

                if isinstance(original_note, list):
                    original_note = ', '.join(original_note)

                if isinstance(new_variation_note, list):
                    new_variation_note = ', '.join(new_variation_note)
                
                if room != room_check1:
                    f.write(r'\hline' + '\n')
                    room_check1 += 1
                    room_repeat_count1 = 0
                
                if room_repeat_count1 == 0:
                    f.write(f'{room} & {location} & {original_note} & {new_variation_note} \\\ ' + '\n')
                    room_repeat_count1 += 1
                elif room_repeat_count1 != 0:
                    f.write(f'{no_repeat_room} & {location} & {original_note} & {new_variation_note} \\\ ' + '\n')

            f.write(r'\hline' + '\n')
            f.write(r'\end{longtable}' + '\n')
            
            room_check2 = 1
            room_repeat_count2 = 0
            f.write(r'\section*{Bass Part Result}' + '\n')
            f.write(r'\begin{longtable}{|c|c|c|c|}' + '\n')
            f.write(r'\hline' + '\n')
            f.write(r'Bar & Note Position & Original Note & New Variation Note \\ \hline' + '\n')
            f.write(r'\endfirsthead' + '\n')
            f.write(r'\multicolumn{4}{c}{{\tablename\ \thetable{} Bass Part Result}} \\' + '\n')
            f.write(r'\hline' + '\n')
            f.write(r'Bar & Note Position & Original Note & New Variation Note \\ \hline' + '\n')
            f.write(r'\hline' + '\n')
            f.write(r'\endhead' + '\n')
            f.write(r'\hline' + '\n')
            f.write(r'\multicolumn{4}{r}{Next page}' + '\n')
            f.write(r'\endfoot' + '\n')
            f.write(r'\hline' + '\n')
            f.write(r'\endlastfoot' + '\n')

            for row in report_list2:
                room = row[0]
                no_repeat_room = " "
                location = row[1]
                original_note = row[2]
                new_variation_note = row[3]

                if isinstance(original_note, list):
                    original_note = ', '.join(original_note)

                if isinstance(new_variation_note, list):
                    new_variation_note = ', '.join(new_variation_note)
                
                if room != room_check2:
                    f.write(r'\hline' + '\n')
                    room_check2 += 1
                    room_repeat_count2 = 0
                if room_repeat_count2 == 0:
                    f.write(f'{room} & {location} & {original_note} & {new_variation_note} \\\ ' + '\n')
                    room_repeat_count2 += 1
                elif room_repeat_count2 != 0:
                    f.write(f'{no_repeat_room} & {location} & {original_note} & {new_variation_note} \\\ ' + '\n')

            f.write(r'\hline' + '\n')
            f.write(r'\end{longtable}' + '\n')
            f.write(r'\end{document}' + '\n')

        process = subprocess.Popen(['xelatex', latex_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        stdout, stderr = process.communicate()

        print("xelatex output:")
        print(stdout.decode())
        print("\nxelatex Result:")
        print(stderr.decode())

        if process.returncode == 0:
            print("Report compilation successful. If you have a table line problem, please make sure to rerun midi2pdfmidi.")
        else:
            print(f"Report compilation failed with return code {process.returncode}.") 
    def expand_note_durations(self, s, divisions):
        for measure in s.getElementsByClass(stream.Measure):
            for element in measure:
                if isinstance(element, music21.note.Note):
                    duration = element.duration.quarterLength
                    expanded_duration = (1 / divisions) * duration
                    offset = element.offset
                    measure.remove(element)
                    for i in range(divisions):
                        new_note = music21.note.Note(element.pitch)
                        new_note.duration.quarterLength = expanded_duration
                        measure.insert(offset + i * expanded_duration, new_note)
                elif isinstance(element, music21.chord.Chord):
                    duration = element.duration.quarterLength
                    expanded_duration = (1 / divisions) * duration
                    offset = element.offset
                    measure.remove(element)
                    for i in range(divisions):
                        new_chord = music21.chord.Chord(element.pitches)
                        for note_in_chord in new_chord:
                            note_in_chord.duration.quarterLength = expanded_duration
                        measure.insert(offset + i * expanded_duration, new_chord)
                elif isinstance(element, music21.note.Rest):
                    duration = element.duration.quarterLength
                    expanded_duration = (1 / divisions) * duration
                    offset = element.offset
                    measure.remove(element)
                    for i in range(divisions):
                        new_rest = music21.note.Rest()
                        new_rest.duration.quarterLength = expanded_duration
                        measure.insert(offset + i * expanded_duration, new_rest)
                        
    def get_scale_from_key(self, key_signature):
        """Return a music21 scale object based on the key signature."""
        tonic = key_signature.tonic
        if key_signature.mode == 'major':
            return scale.MajorScale(tonic)
        elif key_signature.mode == 'minor':
            return scale.MinorScale(tonic)
        else:
            raise ValueError("Unsupported key signature mode")
            
    def variation_maker(self, s, divisions, vt):
        key_signature = s.analyze('key')
        current_scale = self.get_scale_from_key(key_signature)
        for measure in s.getElementsByClass(stream.Measure):
            for element in measure:
                if isinstance(element, music21.note.Note):
                    duration = element.duration.quarterLength
                    expanded_duration = (1 / divisions) * duration
                    offset = element.offset
                    measure.remove(element)
                    next_note = 0
                    for i in range(divisions):
                        new_pitch = element.pitch.transpose(next_note)
                        new_pitch = current_scale.nextPitch(new_pitch)
                        new_note = music21.note.Note()
                        new_note.pitch = new_pitch
                        new_note.duration.quarterLength = expanded_duration
                        measure.insert(offset + i * expanded_duration, new_note)
                        next_note += vt
                elif isinstance(element, music21.chord.Chord):
                    duration = element.duration.quarterLength
                    expanded_duration = (1 / divisions) * duration
                    offset = element.offset
                    measure.remove(element)
                    next_note = 0
                    for i in range(divisions):
                        new_chord = music21.chord.Chord()
                        new_chord.duration.quarterLength = expanded_duration
                        for pitch in element.pitches:
                            new_pitch = pitch.transpose(next_note)
                            new_pitch = current_scale.nextPitch(new_pitch)
                            new_note = music21.note.Note()
                            new_note.pitch = new_pitch
                            new_chord.add(new_note)
                        measure.insert(offset + i * expanded_duration, new_chord)
                        next_note += vt
                elif isinstance(element, music21.note.Rest):
                    duration = element.duration.quarterLength
                    expanded_duration = (1 / divisions) * duration
                    offset = element.offset
                    measure.remove(element)
                    for i in range(divisions):
                        new_rest = music21.note.Rest()
                        new_rest.duration.quarterLength = expanded_duration
                        measure.insert(offset + i * expanded_duration, new_rest)
    
    def midi2pdfmidi(self, seed1=None, seed2=None, T=None, 
                     d=None, r=None, b=None,
                     expand_melody=False, expand_bass=False, divisions=4, 
                     variation_melody=False, variation_bass=False, vt=None,
                     trajstyle=None, 
                     midiname=None, origin_name=None, new_name=None, 
                     report=note_detector, latex_report=report_maker, report_name=None,
                     *args, **kwargs):
        ##############################################################
        # Setting Main Trajectory
        ##############################################################
        
        time, xyz = self.rk4(time=T, ini=seed1, d=d, r=r, b=b, *args, **kwargs)
        x = xyz[:, 0]
        y = xyz[:, 1]
        z = xyz[:, 2]
        new_time, new_xyz = self.rk4(time=T, ini=seed2, d=d, r=r, b=b, *args, **kwargs)
        new_x = new_xyz[:, 0]
        new_y = new_xyz[:, 1]
        new_z = new_xyz[:, 2]
        
        ##############################################################
        # Setting Stream
        ##############################################################
        
        mf = midi.MidiFile()
        mf.open(midiname)
        mf.read()
        mf.close()
        s = midi.translate.midiFileToStream(mf)
        s1 = s[0]
        s2 = s[1]
        ss = midi.translate.midiFileToStream(mf)
        ss1 = ss[0]
        ss2 = ss[1]
        
        if expand_melody == True:
            self.expand_note_durations(s=ss1, divisions=divisions)
        if expand_bass == True:
            self.expand_note_durations(s=ss2, divisions=divisions)
        if variation_melody == True:
            self.variation_maker(s=s1, divisions=divisions, vt=vt)
            self.variation_maker(s=ss1, divisions=divisions, vt=vt)
        if variation_bass == True:
            self.variation_maker(s=s2, divisions=divisions, vt=vt)
            self.variation_maker(s=ss2, divisions=divisions, vt=vt)
        
        ##############################################################
        # Convert Music Data to Python List
        ##############################################################
        
        Melody_Chord_list1 = []
        Duration_list1 = []
        Room_num_list1 = [None]*len(s1)
        Room_index1 = 0
        note_count1 = 0
        Measure_count1 = 1

        for part in s1:
            for n in part.flat.notesAndRests:
                if n.isRest:
                    dur = n.duration.quarterLength
                    note_num = "<REST>"
                    Melody_Chord_list1.append(note_num)
                    Duration_list1.append(dur)
                else:
                    dur = n.duration.quarterLength
                    if len(n.pitches) == 1:
                        note_num = n.pitches[0].midi
                        Melody_Chord_list1.append(note_num)
                        Duration_list1.append(dur)
                    else: 
                        note_num = sorted(set([p.midi for p in n.pitches]))
                        Melody_Chord_list1.append(list(note_num))
                        Duration_list1.append(dur)
                note_count1 += 1
            if part.getElementsByClass(stream.Measure) != Measure_count1:
                Room_num_list1[Room_index1] = note_count1
                note_count1 = 0
                Measure_count1 += 1
                Room_index1 += 1

        Melody_Chord_list2 = []
        Duration_list2 = []
        Room_num_list2 = [None]*len(s2)
        Room_index2 = 0
        note_count2 = 0
        Measure_count2 = 1

        for part in s2:
            for n in part.flat.notesAndRests:
                if n.isRest:
                    dur = n.duration.quarterLength
                    note_num = "<REST>"
                    Melody_Chord_list2.append(note_num)
                    Duration_list2.append(dur)
                else:
                    dur = n.duration.quarterLength
                    if len(n.pitches) == 1:
                        note_num = n.pitches[0].midi
                        Melody_Chord_list2.append(note_num)
                        Duration_list2.append(dur)
                    else: 
                        note_num = sorted(set([p.midi for p in n.pitches]))
                        Melody_Chord_list2.append(list(note_num))
                        Duration_list2.append(dur)
                note_count2 += 1
            if part.getElementsByClass(stream.Measure) != Measure_count2:
                Room_num_list2[Room_index2] = note_count2
                note_count2 = 0
                Measure_count2 += 1
                Room_index2 += 1

        Melody_Chord_list3 = []
        Duration_list3 = []
        Room_num_list3 = [None]*len(ss1)
        Room_index3 = 0
        note_count3 = 0
        Measure_count3 = 1

        for part in ss1:
            for n in part.flat.notesAndRests:
                if n.isRest:
                    dur = n.duration.quarterLength
                    note_num = "<REST>"
                    Melody_Chord_list3.append(note_num)
                    Duration_list3.append(dur)
                else:
                    dur = n.duration.quarterLength
                    if len(n.pitches) == 1:
                        note_num = n.pitches[0].midi
                        Melody_Chord_list3.append(note_num)
                        Duration_list3.append(dur)
                    else: 
                        note_num = sorted(set([p.midi for p in n.pitches]))
                        Melody_Chord_list3.append(list(note_num))
                        Duration_list3.append(dur)
                note_count3 += 1
            if part.getElementsByClass(stream.Measure) != Measure_count3:
                Room_num_list3[Room_index3] = note_count3
                note_count3 = 0
                Measure_count3 += 1
                Room_index3 += 1

        Melody_Chord_list4 = []
        Duration_list4 = []
        Room_num_list4 = [None]*len(ss2)
        Room_index4 = 0
        note_count4 = 0
        Measure_count4 = 1

        for part in ss2:
            for n in part.flat.notesAndRests:
                if n.isRest:
                    dur = n.duration.quarterLength
                    note_num = "<REST>"
                    Melody_Chord_list4.append(note_num)
                    Duration_list4.append(dur)
                else:
                    dur = n.duration.quarterLength
                    if len(n.pitches) == 1:
                        note_num = n.pitches[0].midi
                        Melody_Chord_list4.append(note_num)
                        Duration_list4.append(dur)
                    else: 
                        note_num = sorted(set([p.midi for p in n.pitches]))
                        Melody_Chord_list4.append(list(note_num))
                        Duration_list4.append(dur)
                note_count4 += 1
            if part.getElementsByClass(stream.Measure) != Measure_count4:
                Room_num_list4[Room_index4] = note_count4
                note_count4 = 0
                Measure_count4 += 1
                Room_index4 += 1
                
        ##############################################################
        # Convert Melody and Chord to readable symbol
        ##############################################################
        
        STR_Melody_Chord_list1 = Melody_Chord_list1
        for i in range(len(Melody_Chord_list1)):
            try:
                STR_Melody_Chord_list1[i] = self.note_converter(Melody_Chord_list1[i])
            except TypeError:
                for j in range(len(Melody_Chord_list1[i])):
                    STR_Melody_Chord_list1[i][j] = self.note_converter(Melody_Chord_list1[i][j])
                    
        STR_Melody_Chord_list2 = Melody_Chord_list2
        for i in range(len(Melody_Chord_list2)):
            try:
                STR_Melody_Chord_list2[i] = self.note_converter(Melody_Chord_list2[i])
            except TypeError:
                for j in range(len(Melody_Chord_list2[i])):
                    STR_Melody_Chord_list2[i][j] = self.note_converter(Melody_Chord_list2[i][j])
        
        STR_Melody_Chord_list3 = Melody_Chord_list3
        for i in range(len(Melody_Chord_list3)):
            try:
                STR_Melody_Chord_list3[i] = self.note_converter(Melody_Chord_list3[i])
            except TypeError:
                for j in range(len(Melody_Chord_list3[i])):
                    STR_Melody_Chord_list3[i][j] = self.note_converter(Melody_Chord_list3[i][j])
        
        STR_Melody_Chord_list4 = Melody_Chord_list4
        for i in range(len(Melody_Chord_list4)):
            try:
                STR_Melody_Chord_list4[i] = self.note_converter(Melody_Chord_list4[i])
            except TypeError:
                for j in range(len(Melody_Chord_list4[i])):
                    STR_Melody_Chord_list4[i][j] = self.note_converter(Melody_Chord_list4[i][j])
                    
        ##############################################################
        # Setting Trajectory Mapping Style
        ##############################################################
        
        Criteria_Melody_Chord_list1 = [[index, 
                                       x[::5][index], 
                                       STR_Melody_Chord_list1[index]] 
                                      for index in range(len(STR_Melody_Chord_list1))]
        New_Melody_Chord_list1 = [[index, 
                                  new_x[::5][index]]
                                 for index in range(len(STR_Melody_Chord_list3))]

        Criteria_Melody_Chord_list2 = [[index, 
                                       x[::5][index], 
                                       STR_Melody_Chord_list2[index]] 
                                      for index in range(len(STR_Melody_Chord_list2))]
        New_Melody_Chord_list2 = [[index, 
                                  new_x[::5][index]]
                                 for index in range(len(STR_Melody_Chord_list4))] 
            
        ##############################################################
        # Setting Main Criteria
        ##############################################################
        
        Sorted_Criteria_Melody_Chord_list1 = sorted(Criteria_Melody_Chord_list1, key=lambda x: x[1])
        Sorted_Criteria_Melody_Chord_list2 = sorted(Criteria_Melody_Chord_list2, key=lambda x: x[1])
        
        biindex1 = [i for i in range(0, len(Sorted_Criteria_Melody_Chord_list1))]
        biindex2 = [i for i in range(0, len(Sorted_Criteria_Melody_Chord_list2))]
        if len(Criteria_Melody_Chord_list1) != len(biindex1):
            biindex1 = [i for i in range(0, len(Sorted_Criteria_Melody_Chord_list1) + 1) ]
        if len(Criteria_Melody_Chord_list2) != len(biindex2):
            biindex2 = [i for i in range(0, len(Sorted_Criteria_Melody_Chord_list2) + 1)]
        
        Criteria_Melody_Chord_list1 = [[biindex1[index], 
                                        Sorted_Criteria_Melody_Chord_list1[index][1], 
                                        Sorted_Criteria_Melody_Chord_list1[index][2]] 
                                       for index in range(len(Sorted_Criteria_Melody_Chord_list1))]
        Criteria_Melody_Chord_list2 = [[biindex2[index], 
                                        Sorted_Criteria_Melody_Chord_list2[index][1], 
                                        Sorted_Criteria_Melody_Chord_list2[index][2]] 
                                       for index in range(len(Sorted_Criteria_Melody_Chord_list2))]
        
        ##############################################################
        # Result After Criteria
        ##############################################################
        
        Result1 = [None]*len(New_Melody_Chord_list1)
        for mc in range(len(New_Melody_Chord_list1)):
            for smml in range(len(Criteria_Melody_Chord_list1)):
                if New_Melody_Chord_list1[mc][1] <= Criteria_Melody_Chord_list1[smml][1]: 
                    target_value = Criteria_Melody_Chord_list1[smml][0]
                    Result1[mc] = target_value
                    break
                elif New_Melody_Chord_list1[mc][1] > Criteria_Melody_Chord_list1[-1][1]: 
                    target_value = Criteria_Melody_Chord_list1[-1][0]
                    Result1[mc] = target_value
                    break
        Result2 = [None]*len(New_Melody_Chord_list2)
        for mc in range(len(New_Melody_Chord_list2)):
            for smml in range(len(Criteria_Melody_Chord_list2)):
                if New_Melody_Chord_list2[mc][1] <= Criteria_Melody_Chord_list2[smml][1]:
                    target_value = Criteria_Melody_Chord_list2[smml][0]
                    Result2[mc] = target_value
                    break
                elif New_Melody_Chord_list2[mc][1] > Criteria_Melody_Chord_list2[-1][1]:
                    target_value = Criteria_Melody_Chord_list2[-1][0]
                    Result2[mc] = target_value
                    break
                    
        ##############################################################
        # Check for Out of neighborhood Criteria Note
        ##############################################################
        
        """for check in range(len(Result1)):
            if (Result1[check] == None) and (New_Melody_Chord_list1[check][1] >= Criteria_Melody_Chord_list1[check][1]):
                Result1[check] = max(Criteria_Melody_Chord_list1)[0]
            elif (Result1[check] == None) and (New_Melody_Chord_list1[check][1] <= Criteria_Melody_Chord_list1[check][1]):
                Result1[check] = min(Criteria_Melody_Chord_list1)[0]
        for check in range(len(Result2)):
            if (Result2[check] == None) and (New_Melody_Chord_list2[check][1] >= Criteria_Melody_Chord_list2[check][1]):
                Result2[check] = max(Criteria_Melody_Chord_list2)[0]
            elif (Result2[check] == None) and (New_Melody_Chord_list2[check][1] <= Criteria_Melody_Chord_list2[check][1]):
                Result2[check] = min(Criteria_Melody_Chord_list2)[0]
                
        Result1 = [element + int(len(Sorted_Criteria_Melody_Chord_list1)/2) for element in Result1]
        Result2 = [element + int(len(Sorted_Criteria_Melody_Chord_list2)/2) for element in Result2]
        
        print(Result1)
        
        NV_Melody_Chord_list1 = [Criteria_Melody_Chord_list1[rindex][2] for rindex in Result1]
        NV_Melody_Chord_list2 = [Criteria_Melody_Chord_list2[rindex][2] for rindex in Result2]
        
        replacement_list1 = NV_Melody_Chord_list1
        replacement_list2 = NV_Melody_Chord_list2"""
        
        NV_Melody_Chord_list1 = [Criteria_Melody_Chord_list1[rindex][2] for rindex in Result1]
        NV_Melody_Chord_list2 = [Criteria_Melody_Chord_list2[rindex][2] for rindex in Result2]
        
        replacement_list1 = NV_Melody_Chord_list1
        replacement_list2 = NV_Melody_Chord_list2
        
        ##############################################################
        # Setting duration and Note in new Variation Stream
        ##############################################################
        
        index1 = 0
        for element in ss1.recurse():
            if isinstance(element, (music21.note.Note, music21.chord.Chord, music21.note.Rest)):
                replacement1 = replacement_list1[index1]
                duration_value1 = Duration_list3[index1]
                if replacement1 == '<REST>':
                    new_element = music21.note.Rest()
                elif isinstance(replacement1, list):
                    new_element = music21.chord.Chord(replacement1)
                else:
                    new_element = music21.note.Note(replacement1)

                new_element.duration = music21.duration.Duration(duration_value1)
                element.activeSite.replace(element, new_element)

                index1 += 1
        index2 = 0
        for element in ss2.recurse():
            if isinstance(element, (music21.note.Note, music21.chord.Chord, music21.note.Rest)):
                replacement2 = replacement_list2[index2]
                duration_value2 = Duration_list4[index2]
                if replacement2 == '<REST>':
                    new_element = music21.note.Rest()
                elif isinstance(replacement2, list):
                    new_element = music21.chord.Chord(replacement2)
                else:
                    new_element = music21.note.Note(replacement2)
                
                new_element.duration = music21.duration.Duration(duration_value2)
                element.activeSite.replace(element, new_element)

                index2 += 1
        
        ##############################################################
        # Report Maker
        ##############################################################
        
        """report_melody, report_bass = report(self, 
                                            Room_num_list1=Room_num_list1, 
                                            Room_num_list2=Room_num_list2, 
                                            melody_list=Melody_Chord_list1, 
                                            bass_list=Melody_Chord_list2, 
                                            nv_melody_list=NV_Melody_Chord_list1, 
                                            nv_bass_list=NV_Melody_Chord_list2)
        latex_report(self, report_melody=report_melody, report_bass=report_bass, report_name=report_name)"""
        
        ##############################################################
        # The Last Process
        ##############################################################
        
        us = music21.environment.UserSettings()
        us['lilypondPath'] = self.lilypondPath
        
        #self.expand_note_durations(s=s1, divisions=divisions)
        #self.expand_note_durations(s2)
        
        s.metadata = music21.metadata.Metadata()
        s.metadata.title = origin_name
        conv1 = music21.converter.subConverters.ConverterLilypond()
        scorename = origin_name
        filepath = self.yourfilepath + scorename 
        conv1.write(s, fmt='lilypond', fp=filepath, subformats = ['pdf'])
        sname = origin_name + '.mid'
        s.write('midi', sname)
        
        #s1.show('text')
        #print('-'*30)
        ss.metadata = music21.metadata.Metadata()
        ss.metadata.title = new_name
        conv2 =  music21.converter.subConverters.ConverterLilypond()
        scorename = new_name
        filepath = self.yourfilepath + scorename 
        conv2.write(ss, fmt = 'lilypond', fp=filepath, subformats = ['pdf'])
        ssname = new_name + '.mid'
        ss.write('midi', ssname)
        #ss.show('text')
        
        return "If This function didn't return any error, your file should be in the same .ipynb file directory"

In [5]:
Sound1987 = S1987(lilypondPath="C:\\Users\\helpdesk\\Desktop\\UniverFile\\mingus\\lilypond-2.25.13-mingw-x86_64\\lilypond-2.25.13\\bin\\lilypond.exe",
                  yourfilepath="C:\\Users\\helpdesk\\Desktop\\UniverFile\\Reupload_Final_project\\Program_section\\Class_Development_section\\")

In [6]:
Sound1987.midi2pdfmidi(seed1=np.array([1,1,1]), seed2=np.array([1.01, 1, 1]), T=[0,500], 
                       d=10, r=28, b=8/3,
                       expand_melody=True, expand_bass=True, divisions=4,
                       variation_melody=False, variation_bass=False, vt=2,
                       midiname="Twinkle Twinkle Little Star MIDI.mid", origin_name="test1", new_name="test2")

"If This function didn't return any error, your file should be in the same .ipynb file directory"